In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from datetime import datetime

# Database connection
def create_db_engine():
    
    db_url = "__________________"
    return create_engine(db_url)

# SQL queries
queries = {
    "date_range": 
        SELECT MIN(date) AS start_date, MAX(date) AS end_date, 
               COUNT(DISTINCT date) AS unique_invoice_days
        FROM date_dim;
    ,
    "profit_by_day": 
        SELECT dd.date, SUM(sf.gross_income) AS total_gross_income
        FROM sales_fact sf
        JOIN date_dim dd ON sf.date_id = dd.date_id
        GROUP BY dd.date
        ORDER BY total_gross_income;
    ,
    "branch_analysis": 
        SELECT jd.branch, COUNT(DISTINCT sf.order_id) AS invoice_count,
               SUM(sf.gross_income) AS total_gross_income
        FROM sales_fact sf
        JOIN junk_dim jd ON sf.junk_id = jd.junk_id
        GROUP BY jd.branch
        ORDER BY total_gross_income DESC;
    ,
    "customer_payment": 
        SELECT jd.customer_type, COUNT(*) AS count
        FROM junk_dim jd
        GROUP BY jd.customer_type
        UNION ALL
        SELECT od.payment_method, COUNT(*) AS count
        FROM order_dim od
        GROUP BY od.payment_method;
    ,
    "product_rating": 
        SELECT od.product_line, COUNT(*) AS count, AVG(od.rating) AS avg_rating
        FROM order_dim od
        GROUP BY od.product_line
        ORDER BY count DESC;
    ,
    "gender_quantity": 
        SELECT jd.customer_sex AS gender, SUM(sf.quantity) AS total_quantity
        FROM sales_fact sf
        JOIN junk_dim jd ON sf.junk_id = jd.junk_id
        GROUP BY jd.customer_sex;
    ,
    "product_gender":
        SELECT od.product_line, jd.customer_sex AS gender, 
               SUM(sf.quantity) AS total_quantity
        FROM sales_fact sf
        JOIN order_dim od ON sf.order_id = od.order_id
        JOIN junk_dim jd ON sf.junk_id = jd.junk_id
        GROUP BY od.product_line, jd.customer_sex
        ORDER BY od.product_line, total_quantity DESC;
    ,
    "sales_trend": 
        SELECT dd.date, SUM(sf.Total) AS daily_sales
        FROM sales_fact sf
        JOIN date_dim dd ON sf.date_id = dd.date_id
        GROUP BY dd.date
        ORDER BY dd.date;
    ,
    "revenue_product": 
        SELECT od.product_line, SUM(sf.Total) AS total_revenue
        FROM sales_fact sf
        JOIN order_dim od ON sf.order_id = od.order_id
        GROUP BY od.product_line
        ORDER BY total_revenue DESC;
    
}



In [ ]:
def run_query(engine, query):
    return pd.read_sql_query(query, engine)

In [ ]:
def date_range_analysis(df):
    print("Date Range Analysis:")
    print(f"Start Date: {df['start_date'].iloc[0]}")
    print(f"End Date: {df['end_date'].iloc[0]}")
    print(f"Number of unique invoice days: {df['unique_invoice_days'].iloc[0]}")

def profit_analysis(df):
    plt.figure(figsize=(12, 6))
    plt.plot(df['date'], df['total_gross_income'])
    plt.title('Daily Gross Income')
    plt.xlabel('Date')
    plt.ylabel('Gross Income')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def branch_analysis(df):
    plt.figure(figsize=(10, 6))
    sns.barplot(x='branch', y='total_gross_income', data=df)
    plt.title('Total Gross Income by Branch')
    plt.xlabel('Branch')
    plt.ylabel('Total Gross Income')
    plt.show()

    plt.figure(figsize=(10, 6))
    sns.barplot(x='branch', y='invoice_count', data=df)
    plt.title('Invoice Count by Branch')
    plt.xlabel('Branch')
    plt.ylabel('Invoice Count')
    plt.show()

def customer_payment_analysis(df):
    customer_types = df[df['customer_type'].isin(['Member', 'Normal'])]
    payment_methods = df[~df['customer_type'].isin(['Member', 'Normal'])]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    sns.barplot(x='customer_type', y='count', data=customer_types, ax=ax1)
    ax1.set_title('Customer Type Distribution')
    ax1.set_xlabel('Customer Type')
    ax1.set_ylabel('Count')

    sns.barplot(x='customer_type', y='count', data=payment_methods, ax=ax2)
    ax2.set_title('Payment Method Distribution')
    ax2.set_xlabel('Payment Method')
    ax2.set_ylabel('Count')

    plt.tight_layout()
    plt.show()

def product_rating_analysis(df):
    plt.figure(figsize=(12, 6))
    sns.barplot(x='product_line', y='count', data=df)
    plt.title('Product Line Distribution')
    plt.xlabel('Product Line')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(12, 6))
    sns.barplot(x='product_line', y='avg_rating', data=df)
    plt.title('Average Rating by Product Line')
    plt.xlabel('Product Line')
    plt.ylabel('Average Rating')
    plt.xticks(rotation=45)
    plt.show()

def gender_quantity_analysis(df):
    plt.figure(figsize=(8, 6))
    sns.barplot(x='gender', y='total_quantity', data=df)
    plt.title('Total Quantity Purchased by Gender')
    plt.xlabel('Gender')
    plt.ylabel('Total Quantity')
    plt.show()

def product_gender_analysis(df):
    plt.figure(figsize=(12, 6))
    sns.barplot(x='product_line', y='total_quantity', hue='gender', data=df)
    plt.title('Product Line Purchases by Gender')
    plt.xlabel('Product Line')
    plt.ylabel('Total Quantity')
    plt.xticks(rotation=45)
    plt.legend(title='Gender')
    plt.show()

def sales_trend_analysis(df):
    df['date'] = pd.to_datetime(df['date'])
    plt.figure(figsize=(12, 6))
    plt.plot(df['date'], df['daily_sales'])
    plt.title('Sales Trend Over Time')
    plt.xlabel('Date')
    plt.ylabel('Daily Sales')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def revenue_product_analysis(df):
    plt.figure(figsize=(12, 6))
    sns.barplot(x='product_line', y='total_revenue', data=df)
    plt.title('Total Revenue by Product Line')
    plt.xlabel('Product Line')
    plt.ylabel('Total Revenue')
    plt.xticks(rotation=45)
    plt.show()



In [ ]:
def main():
    engine = create_db_engine()
    
    # Run analyses
    date_range_analysis(run_query(engine, queries['date_range']))
    profit_analysis(run_query(engine, queries['profit_by_day']))
    branch_analysis(run_query(engine, queries['branch_analysis']))
    customer_payment_analysis(run_query(engine, queries['customer_payment']))
    product_rating_analysis(run_query(engine, queries['product_rating']))
    gender_quantity_analysis(run_query(engine, queries['gender_quantity']))
    product_gender_analysis(run_query(engine, queries['product_gender']))
    sales_trend_analysis(run_query(engine, queries['sales_trend']))
    revenue_product_analysis(run_query(engine, queries['revenue_product']))

if __name__ == "__main__":
    main()